In [144]:
import pandas as pd
import numpy as np
import vertica_python
import os
import datetime as dt
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from math import sqrt
from sklearn.model_selection import train_test_split
import _pickle as cPickle
import time


In [145]:
os.chdir('/home/centos/notebooks/Shashank/rto_testing/')

### Reading data 

In [5]:
col_names = ['code', 'order_created_date', 'address_length', 'address_has_numeric', 'subo_shipping_method_code',
				'dest_pincode', 'subcategory_id', 
				'bucket_id', 'user_del', 'user_del_ship', 
				'city_att_14', 'city_del_14', 'pin_att_14', 'pin_del_14',
				'courier_group', 'rto', 'Same_SUPC_Dup', 
				'Same_Subcat_Dup', 'ITR', 'last_terminal_status', 'last_order_payment_mode', 'last_order_promo',
				'last_order_rto_prob', 'ever_std', 'ever_rating', 'ever_nps', 'ever_rex', 'Dest_City', 'dest_state']

select_vars = ','.join(col_names)


In [6]:
conn_info = dict(host='10.65.0.201', port=5433, database='snapdealdwh', user = 'datascience', password = 'D@t@$c!#9786')

conn = vertica_python.connect(**conn_info)
cur = conn.cursor()
query_fetch = "select %s from analytics_logistics.ys_jsfull_data_fin_new1 where shipped = 1;" % (select_vars)
cur.execute(query_fetch)
data = cur.fetchall()
conn.close()

mydata = pd.DataFrame(data = data, columns = col_names)
mydata.to_csv('rto_data_july_sept21.csv', index = False)


In [11]:
col_names = ["suborder_code", "cg_dp_attempt14", "cg_dp_del14", "cg_dc_attempt14", "cg_dc_del14", 
                "cg_ds_attempt14", "cg_ds_del14", "cg_dp_o2a", "cg_dc_o2a", "cg_ds_o2a", 
                "cg_dp_breach", "cg_dc_breach", "cg_ds_breach", "cg_dp_pendency", 
                "cg_dc_pendency", "cg_ds_pendency", "last_del_diff", "pog_orders10", 
                "pog_cancelled10", "bucket_attempt14", "bucket_del14", "cbpu3", "cbpu1",
                "subcat_attempt14", "subcat_del14", "bucket_cbpu0", "overall_cbpu0", 
                "order_time", "estimated_delivery", "paid_amount", "promo_discount"]

select_vars = ','.join(col_names)


In [12]:
conn_info = dict(host='10.65.0.201', port=5433, database='snapdealdwh', user = 'datascience', password = 'D@t@$c!#9786')

conn = vertica_python.connect(**conn_info)
cur = conn.cursor()
query_fetch = "select %s from analytics_logistics.rto_model_new_secdata;" % (select_vars)
cur.execute(query_fetch)
data = cur.fetchall()
conn.close()

courier_data = pd.DataFrame(data = data, columns = col_names)
courier_data.to_csv('courier_data_july_sept21.csv', index = False)


### Data Cleaning

In [63]:
courier_data = pd.read_csv("courier_data_july_sept21.csv")
courier_data = courier_data.rename(columns = {'suborder_code': 'code'})
courier_data.head(2)

,code,cg_dp_attempt14,cg_dp_del14,cg_dc_attempt14,cg_dc_del14,cg_ds_attempt14,cg_ds_del14,cg_dp_o2a,cg_dc_o2a,cg_ds_o2a,...,cbpu3,cbpu1,subcat_attempt14,subcat_del14,bucket_cbpu0,overall_cbpu0,order_time,estimated_delivery,paid_amount,promo_discount
0,47453908959,26.0,19.0,6034.0,3827.0,11520.0,6841.0,5.61,6.29,6.97,...,57.50,60.56,3772.0,2434.0,59.75,76.11,2021-08-14 17:12:50,2021-08-22 17:12:50,169,0.0
1,48081795737,46.0,38.0,16904.0,10705.0,25308.0,15365.0,6.27,6.40,6.77,...,71.38,73.98,5062.0,3258.0,78.20,99.87,2021-09-18 17:05:03,2021-09-29 17:05:03,145,0.0


In [64]:
courier_data['order_date_time'] = pd.to_datetime(courier_data['order_time'], infer_datetime_format=True) 
courier_data['order_date'] = courier_data['order_date_time'].dt.date
courier_data['del_date_time'] = pd.to_datetime(courier_data['estimated_delivery'], infer_datetime_format=True) 
courier_data['del_date'] = courier_data['del_date_time'].dt.date
courier_data.head(2)

,code,cg_dp_attempt14,cg_dp_del14,cg_dc_attempt14,cg_dc_del14,cg_ds_attempt14,cg_ds_del14,cg_dp_o2a,cg_dc_o2a,cg_ds_o2a,...,bucket_cbpu0,overall_cbpu0,order_time,estimated_delivery,paid_amount,promo_discount,order_date_time,order_date,del_date_time,del_date
0,47453908959,26.0,19.0,6034.0,3827.0,11520.0,6841.0,5.61,6.29,6.97,...,59.75,76.11,2021-08-14 17:12:50,2021-08-22 17:12:50,169,0.0,2021-08-14 17:12:50,2021-08-14,2021-08-22 17:12:50,2021-08-22
1,48081795737,46.0,38.0,16904.0,10705.0,25308.0,15365.0,6.27,6.40,6.77,...,78.20,99.87,2021-09-18 17:05:03,2021-09-29 17:05:03,145,0.0,2021-09-18 17:05:03,2021-09-18,2021-09-29 17:05:03,2021-09-29


In [65]:
courier_data['promise'] = (courier_data['del_date'] - courier_data['order_date']).dt.days
courier_data.head(2)

,code,cg_dp_attempt14,cg_dp_del14,cg_dc_attempt14,cg_dc_del14,cg_ds_attempt14,cg_ds_del14,cg_dp_o2a,cg_dc_o2a,cg_ds_o2a,...,overall_cbpu0,order_time,estimated_delivery,paid_amount,promo_discount,order_date_time,order_date,del_date_time,del_date,promise
0,47453908959,26.0,19.0,6034.0,3827.0,11520.0,6841.0,5.61,6.29,6.97,...,76.11,2021-08-14 17:12:50,2021-08-22 17:12:50,169,0.0,2021-08-14 17:12:50,2021-08-14,2021-08-22 17:12:50,2021-08-22,8
1,48081795737,46.0,38.0,16904.0,10705.0,25308.0,15365.0,6.27,6.40,6.77,...,99.87,2021-09-18 17:05:03,2021-09-29 17:05:03,145,0.0,2021-09-18 17:05:03,2021-09-18,2021-09-29 17:05:03,2021-09-29,11


In [66]:
mydata = pd.read_csv('rto_data_july_sept21.csv')
mydata = mydata.rename(columns = {'address_length': 'add_length', 'address_has_numeric': 'is_num'})
mydata['order_created_date'] = pd.to_datetime(mydata['order_created_date'], infer_datetime_format=True) 
mydata['order_created_date'] = mydata['order_created_date'].dt.date
mydata['is_num'] = np.where(mydata['is_num'] == 'Yes', 1, 0)
mydata['add_length'] = mydata['add_length'].astype('float')
mydata['user_del_ship'] = mydata['user_del_ship'].replace(0, np.nan)
mydata.head(2)

,code,order_created_date,add_length,is_num,subo_shipping_method_code,dest_pincode,subcategory_id,bucket_id,user_del,user_del_ship,...,last_terminal_status,last_order_payment_mode,last_order_promo,last_order_rto_prob,ever_std,ever_rating,ever_nps,ever_rex,Dest_City,dest_state
0,46824406930,2021-07-01,64.0,0,COD,416507,160,5502,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOLHAPUR,MAHARASHTRA
1,46824414550,2021-07-01,29.0,1,COD,502270,178,1244,NaN,NaN,...,NaN,COD,0.0,0.411999,0.0,0.0,0.0,0.0,JOGIPET,TELANGANA


In [67]:
mydata.shape

(12029885, 29)

In [68]:
mydata['code'] = mydata['code'].astype('str')
courier_data['code'] = courier_data['code'].astype('str')

In [69]:
mydata = pd.merge(mydata, courier_data, how = 'inner', on = 'code')

In [75]:
mydata = mydata.loc[mydata['subo_shipping_method_code'] == 'COD']

In [76]:
mydata.shape

(10951606, 64)

### Dividing in Train, Test, Validation and feature creation

In [96]:
# dividing data in train and test

train = mydata.loc[(mydata['order_created_date'] >= dt.date(2021, 8, 10)) & (mydata['order_created_date'] <= dt.date(2021, 9, 10)),:]
# valid = mydata.loc[(mydata['order_created_date'] > dt.date(2021, 6, 15)) & (mydata['order_created_date'] <= dt.date(2021, 6, 20),:]
test = mydata.loc[mydata['order_created_date'] > dt.date(2021, 9, 10),:]

# creating additional variables

train = train.drop(columns = ['order_created_date', 'code'])
train = train.dropna(subset = ['subcategory_id', 'bucket_id'])
train[['subcategory_id', 'bucket_id']] = train[['subcategory_id', 'bucket_id']].astype('int64')

cols = ['user_del_ship', 'pin_att_14', 'city_att_14', 'cg_dp_attempt14', 'cg_dc_attempt14', 'cg_ds_attempt14', 'pog_orders10', 'bucket_attempt14', 'subcat_attempt14']
train[cols] = train[cols].replace(0,np.nan)

train.loc[train['user_del_ship'].isna(), 'user_del'] = np.nan

train['del_per'] = np.where(train['user_del_ship'] != 0, train['user_del']*100/train['user_del_ship'], np.nan)

train['cg_dp_fasr_14'] = np.where(~train['cg_dp_attempt14'].isna(), train['cg_dp_del14']*100/train['cg_dp_attempt14'], np.nan)
train['cg_dc_fasr_14'] = np.where(~train['cg_dc_attempt14'].isna(), train['cg_dc_del14']*100/train['cg_dc_attempt14'], np.nan)
train['cg_ds_fasr_14'] = np.where(~train['cg_ds_attempt14'].isna(), train['cg_ds_del14']*100/train['cg_ds_attempt14'], np.nan)

train['bucket_fasr_14'] = np.where(~train['bucket_attempt14'].isna(), train['bucket_del14']*100/train['bucket_attempt14'], np.nan)
train['subcat_fasr_14'] = np.where(~train['subcat_attempt14'].isna(), train['subcat_del14']*100/train['subcat_attempt14'], np.nan)
train['pog_cancel'] = np.where(~train['pog_orders10'].isna(), train['pog_cancelled10']*100/train['pog_orders10'], np.nan)

train['pin_fasr_14'] = np.where(~train['pin_att_14'].isna(), train['pin_del_14']*100/train['pin_att_14'], np.nan)
train['city_fasr_14'] = np.where(~train['city_att_14'].isna(), train['city_del_14']*100/train['city_att_14'], np.nan)

train[['Same_SUPC_Dup', 'Same_Subcat_Dup']] = train[['Same_SUPC_Dup', 'Same_Subcat_Dup']].fillna(0)
train['ITR'] = train['ITR'].fillna(0)

train['same_supc'] = np.where(train['Same_SUPC_Dup'] > 0, 1, 0)
train['same_subcat'] = np.where(train['Same_Subcat_Dup'] > 0, 1, 0)

train['three_80'] = np.where((train['user_del_ship'] > 3) & (train['del_per'] < 20), 1, 0)
train['five_70'] = np.where((train['user_del_ship'] > 5) & (train['del_per'] < 30), 1, 0)

train.loc[train['user_del_ship'].isna(), 'three_80'] = np.nan
train.loc[train['user_del_ship'].isna(), 'five_70'] = np.nan


top_vars = ['del_per', 'pin_fasr_14',
				'add_length', 'city_fasr_14', 'dest_pincode', 'user_del',
				'is_num', 'same_supc', 'same_subcat', 'ITR', 'three_80', 'five_70', 'user_del_ship', 
				'last_terminal_status', 'cg_dp_fasr_14', 'cg_dc_fasr_14', 'cg_ds_fasr_14', 'cg_dp_o2a', 
				'cg_dc_breach', 'cg_ds_breach', 'cg_dp_pendency', 'cg_dc_pendency', 'cg_dc_o2a', 'cg_ds_o2a', 'cg_dp_breach',
				'cg_ds_pendency', 'last_del_diff', 'courier_group', 'pog_cancel', 'bucket_fasr_14', 'cbpu3', 'cbpu1', 
				'subcat_fasr_14', 'bucket_cbpu0', 'overall_cbpu0', 'promise', 
				'promo_discount', 'paid_amount']

train = train[top_vars + ['rto']]


In [97]:

test = test.dropna(subset = ['subcategory_id', 'bucket_id'])
test[['subcategory_id', 'bucket_id']] = test[['subcategory_id', 'bucket_id']].astype('int64')

test[cols] = test[cols].replace(0,np.nan)

test.loc[test['user_del_ship'].isna(), 'user_del'] = np.nan

test['del_per'] = np.where(test['user_del_ship'] != 0, test['user_del']*100/test['user_del_ship'], np.nan)

test['cg_dp_fasr_14'] = np.where(~test['cg_dp_attempt14'].isna(), test['cg_dp_del14']*100/test['cg_dp_attempt14'], np.nan)
test['cg_dc_fasr_14'] = np.where(~test['cg_dc_attempt14'].isna(), test['cg_dc_del14']*100/test['cg_dc_attempt14'], np.nan)
test['cg_ds_fasr_14'] = np.where(~test['cg_ds_attempt14'].isna(), test['cg_ds_del14']*100/test['cg_ds_attempt14'], np.nan)

test['bucket_fasr_14'] = np.where(~test['bucket_attempt14'].isna(), test['bucket_del14']*100/test['bucket_attempt14'], np.nan)
test['subcat_fasr_14'] = np.where(~test['subcat_attempt14'].isna(), test['subcat_del14']*100/test['subcat_attempt14'], np.nan)
test['pog_cancel'] = np.where(~test['pog_orders10'].isna(), test['pog_cancelled10']*100/test['pog_orders10'], np.nan)

test['pin_fasr_14'] = np.where(~test['pin_att_14'].isna(), test['pin_del_14']*100/test['pin_att_14'], np.nan)
test['city_fasr_14'] = np.where(~test['city_att_14'].isna(), test['city_del_14']*100/test['city_att_14'], np.nan)

test[['Same_SUPC_Dup', 'Same_Subcat_Dup']] = test[['Same_SUPC_Dup', 'Same_Subcat_Dup']].fillna(0)
test['ITR'] = test['ITR'].fillna(0)

test['same_supc'] = np.where(test['Same_SUPC_Dup'] > 0, 1, 0)
test['same_subcat'] = np.where(test['Same_Subcat_Dup'] > 0, 1, 0)

test['three_80'] = np.where((test['user_del_ship'] > 3) & (test['del_per'] < 20), 1, 0)
test['five_70'] = np.where((test['user_del_ship'] > 5) & (test['del_per'] < 30), 1, 0)

test.loc[test['user_del_ship'].isna(), 'three_80'] = np.nan
test.loc[test['user_del_ship'].isna(), 'five_70'] = np.nan


In [98]:
cat_vars = ['last_terminal_status', 'courier_group']

train[cat_vars] = train[cat_vars].astype('str')
test[cat_vars] = test[cat_vars].astype('str')


In [99]:
# combining long tail of categorical variables

# club_vars = ['subcategory_id', 'bucket_id', 'dest_pincode', 'Dest_City', 'dest_state']

def club_categories(x,n = 100):
    freq = x.value_counts()
    less_freq_cats = list(freq[freq < n].keys())
    x[x.isin(less_freq_cats)] = 'other'
    return(x)

train[cat_vars] = train[cat_vars].apply(club_categories, axis = 0)

for var in cat_vars:
	train_values = train[var].unique()
	test.loc[~test[var].isin(train_values), var] = 'other'


# replacing categorical data with weight of evidence of respective categories

def iv_variable(data, indep, dep):

	sub_data = data[[indep, dep]].copy()
	event = sub_data[dep].sum()
	non_event = sub_data.shape[0] - event
	iv_data = sub_data.groupby(indep).agg(['sum', 'size']).rename(columns = {'sum': 'event', 'size': 'total'})
	iv_data.columns = iv_data.columns.droplevel(level=0)
	iv_data = iv_data.reset_index(drop = False)
	iv_data['non_event'] = iv_data['total'] - iv_data['event'] 
	iv_data['per_event'] = iv_data['event']/event
	iv_data['per_non_event'] = iv_data['non_event']/non_event
	iv_data['woe'] = np.log(((iv_data['non_event'] + 0.5)/non_event)/((iv_data['event'] + 0.5)/event)) * (iv_data['per_non_event'] - iv_data['per_event'])
	iv_data['woe'] = 100*iv_data['woe']
	iv_data['cat_rto'] = iv_data['event']/iv_data['total']
	base_rto = event/(event + non_event)
	iv_data['woe'] = np.where(iv_data['cat_rto'] < base_rto, iv_data['woe']*(-1.0), iv_data['woe'])
	iv_data = iv_data[[indep, 'woe']]
	return(iv_data)


woe_dict = {}

start = time.time()

for var in cat_vars:
	iv_var = iv_variable(train, var, 'rto')
	woe_dict[var] = dict(zip(iv_var[var], iv_var['woe']))
	train = pd.merge(train, iv_var, how = 'left', on = var).drop(columns = [var]).rename(columns = {'woe': var})
	test = pd.merge(test, iv_var, how = 'left', on = var).drop(columns = [var]).rename(columns = {'woe': var})

end = time.time()


In [100]:
valid = test.loc[test['order_created_date'] <= dt.date(2021, 9, 15),:]
test = test.loc[test['order_created_date'] > dt.date(2021, 9, 15),:]


In [101]:
out_of_time = test[['order_created_date', 'code', 'rto']].copy()
valid = valid[top_vars + ['rto']]
test = test[top_vars + ['rto']]

In [102]:
valid.head(2)

,del_per,pin_fasr_14,add_length,city_fasr_14,dest_pincode,user_del,is_num,same_supc,same_subcat,ITR,...,bucket_fasr_14,cbpu3,cbpu1,subcat_fasr_14,bucket_cbpu0,overall_cbpu0,promise,promo_discount,paid_amount,rto
0,NaN,42.473118,29.0,45.477387,481880,NaN,1,0,0,0.0,...,61.078622,66.52,69.05,59.104781,57.99,62.68,10,0.0,199,1
1,83.333333,69.811321,56.0,60.932910,212214,5.0,1,0,1,1.0,...,61.618696,66.20,65.86,61.619419,72.30,62.68,7,0.0,575,0


In [103]:
# imputing missing values with a fixed large negative numeric value so that it will be treated as a different category

train.fillna(-99999, inplace = True)
test.fillna(-99999, inplace = True)
valid.fillna(-99999, inplace = True)


In [147]:
train.head(2)

,del_per,pin_fasr_14,add_length,city_fasr_14,dest_pincode,user_del,is_num,same_supc,same_subcat,ITR,...,cbpu1,subcat_fasr_14,bucket_cbpu0,overall_cbpu0,promise,promo_discount,paid_amount,rto,last_terminal_status,courier_group
0,100.0,64.838710,25.0,62.195122,451001,1.0,1,0,0,0.0,...,112.7,54.57485,113.87,60.4,12,0.0,499,1,-9.133036,0.152875
1,-99999.0,41.176471,49.0,65.677147,423213,-99999.0,0,0,0,0.0,...,59.6,62.75575,67.83,60.4,3,0.0,539,1,0.319413,0.070209


In [148]:
model_vars = ['del_per',
                 'last_terminal_status',
                 'add_length',
                 'pin_fasr_14',
                 'city_fasr_14',
                 'user_del',
                 'ITR',
                 'same_subcat',
                 'user_del_ship',
                 'same_supc',
                 'five_70',
                 'three_80',
                 'last_del_diff',
                 'is_num',
                 'pog_cancel',
                 'bucket_fasr_14', 
                 'subcat_fasr_14' 
]

In [149]:
# dividing training into training and validation data

train_x, train_y = train[model_vars], train['rto']
valid_x, valid_y = valid[model_vars], valid['rto']

test_x, test_y = test[model_vars], test['rto']


In [154]:
# training the model

xgb_clf = xgb.XGBClassifier(max_depth = 6,
                           min_child_weight = 1,
                           learning_rate = 0.1,
                           n_estimators = 200,
                           objective = 'binary:logistic',
                           gamma = 5,
                           n_jobs = 30,
                           use_label_encoder=False)


In [155]:
xgb_clf.fit(train_x, train_y, eval_metric = 'error', 
            eval_set = [(train_x, train_y), (valid_x, valid_y)], early_stopping_rounds = 50)


[0]	validation_0-error:0.28611	validation_1-error:0.28541
[1]	validation_0-error:0.28500	validation_1-error:0.28460
[2]	validation_0-error:0.28483	validation_1-error:0.28407
[3]	validation_0-error:0.28422	validation_1-error:0.28372
[4]	validation_0-error:0.28435	validation_1-error:0.28366
[5]	validation_0-error:0.28412	validation_1-error:0.28336
[6]	validation_0-error:0.28386	validation_1-error:0.28300
[7]	validation_0-error:0.28392	validation_1-error:0.28299
[8]	validation_0-error:0.28370	validation_1-error:0.28287
[9]	validation_0-error:0.28335	validation_1-error:0.28267
[10]	validation_0-error:0.28312	validation_1-error:0.28238
[11]	validation_0-error:0.28299	validation_1-error:0.28232
[12]	validation_0-error:0.28257	validation_1-error:0.28197
[13]	validation_0-error:0.28235	validation_1-error:0.28143
[14]	validation_0-error:0.28189	validation_1-error:0.28109
[15]	validation_0-error:0.28178	validation_1-error:0.28103
[16]	validation_0-error:0.28142	validation_1-error:0.28082
[17]	va

[139]	validation_0-error:0.27630	validation_1-error:0.27730
[140]	validation_0-error:0.27630	validation_1-error:0.27729
[141]	validation_0-error:0.27630	validation_1-error:0.27729
[142]	validation_0-error:0.27630	validation_1-error:0.27728
[143]	validation_0-error:0.27629	validation_1-error:0.27728
[144]	validation_0-error:0.27629	validation_1-error:0.27730
[145]	validation_0-error:0.27628	validation_1-error:0.27730
[146]	validation_0-error:0.27628	validation_1-error:0.27730
[147]	validation_0-error:0.27627	validation_1-error:0.27729
[148]	validation_0-error:0.27627	validation_1-error:0.27728
[149]	validation_0-error:0.27626	validation_1-error:0.27728
[150]	validation_0-error:0.27626	validation_1-error:0.27730
[151]	validation_0-error:0.27626	validation_1-error:0.27729
[152]	validation_0-error:0.27627	validation_1-error:0.27731
[153]	validation_0-error:0.27626	validation_1-error:0.27731
[154]	validation_0-error:0.27626	validation_1-error:0.27731
[155]	validation_0-error:0.27625	validat

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=5, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=30, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [156]:
# getting variable importance

var_imp = xgb_clf.get_booster().get_score(importance_type = 'total_gain')
imp_data = pd.DataFrame({'feature': list(var_imp.keys()), 'importance': list(var_imp.values())})
imp_data = imp_data.sort_values(by = 'importance', ascending = False)
imp_data.importance = imp_data.importance/np.sum(imp_data.importance)
imp_data.reset_index(drop = True, inplace = True)
imp_data

,feature,importance
0,del_per,0.412018
1,last_terminal_status,0.135689
2,pin_fasr_14,0.105053
3,bucket_fasr_14,0.085972
4,add_length,0.063829
5,user_del,0.034967
6,city_fasr_14,0.032349
7,same_subcat,0.026922
8,ITR,0.026236
9,same_supc,0.017483


In [157]:
# getting test accuracy 

predicted_values = xgb_clf.predict_proba(test_x)[:,1]
predicted_class = np.where(predicted_values > 0.5, 1, 0)
test_accuracy = accuracy_score(test_y, predicted_class)
test_lgloss = log_loss(test_y, predicted_values)

print('test accuracy: %.3f, test logloss: %.3f' % (test_accuracy, test_lgloss))

test accuracy: 0.742, test logloss: 0.531


In [158]:

out_of_time['rto_prob'] = predicted_values
# out_of_time.to_csv('rto_probs_sept21.csv', index = False)
# out_of_time = pd.read_csv('rto_probs_sept21.csv')

out_of_time['decile'] = pd.cut(out_of_time['rto_prob'], bins = np.arange(0,1.01, 0.1))
# out_of_time['decile'] = out_of_time['decile'].astype('str').str.replace(' ', '')

out = out_of_time.groupby(['decile'])['rto'].agg(['size', 'sum']).reset_index(drop = False).rename(columns = {'size': 'count', 'sum': 'rto'})
out['rto_per'] = out['rto']/out['count']
out['salience'] = out['count']/out_of_time.shape[0]
out

,decile,count,rto,rto_per,salience
0,"(0.0, 0.1]",168086,11160,0.066395,0.075385
1,"(0.1, 0.2]",560429,78180,0.139500,0.251347
2,"(0.2, 0.3]",560097,127463,0.227573,0.251198
3,"(0.3, 0.4]",400665,128189,0.319941,0.179695
4,"(0.4, 0.5]",262163,108817,0.415074,0.117578
5,"(0.5, 0.6]",168362,87158,0.517682,0.075509
6,"(0.6, 0.7]",86595,53527,0.618130,0.038837
7,"(0.7, 0.8]",21388,15103,0.706144,0.009592
8,"(0.8, 0.9]",1534,1264,0.823990,0.000688
9,"(0.9, 1.0]",381,355,0.931759,0.000171


In [159]:
sal_g6 = out.loc[out['rto_per'] > 0.6, 'salience'].sum()*100
rto_g6 = out.loc[out['rto_per'] > 0.6, 'rto'].sum()*100/out.loc[out['rto_per'] > 0.6, 'count'].sum()

sal_g7 = out.loc[out['rto_per'] > 0.7, 'salience'].sum()*100
rto_g7 = out.loc[out['rto_per'] > 0.7, 'rto'].sum()*100/out.loc[out['rto_per'] > 0.7, 'count'].sum()

print('sal > 0.6: %.2f, rto > 0.6: %.2f, sal > 0.7: %.2f, rto > 0.7: %.2f' % (sal_g6, rto_g6, sal_g7, rto_g7))


sal > 0.6: 4.93, rto > 0.6: 63.92, sal > 0.7: 1.05, rto > 0.7: 71.76


In [160]:
test_x.columns

Index(['del_per', 'last_terminal_status', 'add_length', 'pin_fasr_14',
       'city_fasr_14', 'user_del', 'ITR', 'same_subcat', 'user_del_ship',
       'same_supc', 'five_70', 'three_80', 'last_del_diff', 'is_num',
       'pog_cancel', 'bucket_fasr_14', 'subcat_fasr_14'],
      dtype='object')

In [161]:
woe_dict

{'last_terminal_status': {'DEL': -9.133036128016919,
  'RTO': 11.515644659892194,
  'nan': 0.3194134552522803},
 'courier_group': {'BLUEDART': 0.15287468657466227,
  'DELHIVERY': 0.07020943111854007,
  'ECOMM_EXP': 0.13882741708316065,
  'EKART': 0.003989783448206044,
  'FEDEX': 0.0029582277495173045,
  'GATI': 0.07197671689298032,
  'LAAP_DOP': 0.08473264865822945,
  'LAAP_SD': -1.6550316044615214,
  'SHADOWFAX': -0.06557309474373016,
  'XPRESSBEES': 0.4518519648778564}}

In [162]:
## Saving model and woe dict

xgb_clf.save_model('rto_model_1021.json')

f = open('woe_dict_1021.pkl','wb')
cPickle.dump(woe_dict,f,-1)
f.close()
